In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import tensorflow as tf
from xgboost import XGBRegressor as xgb
from sklearn.model_selection import train_test_split

In [2]:
def build_model(activation, dropout_rate=0.0, n_hidden=128):
    print("Building model with {} activation and {:.3f} dropout".format(activation, dropout_rate))
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(25,)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(n_hidden, activation=activation),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(n_hidden, activation=activation),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, activation=tf.identity)
    ])
    sgd = tf.keras.optimizers.Adam(lr=1e-3)#, decay=1e-6, momentum=0.9, nesterov=True)
    if activation == 'sigmoid':
        model.compile(
                      loss='MSE',
                      metrics=['mse'],
                     optimizer=sgd)
    else:
        model.compile(
              loss='MSE',
              metrics=['mse'],
                     optimizer=sgd)
    return model

In [6]:
dataset_iter = 0
max_epoch = 500
with open("results/results_over_epoch_{}.tsv".format(max_epoch), 'w') as results_file:
    for add_amount, mult_amount in zip([1.0, 0.0], [0.0, 1.0]):
        X = np.random.uniform(-1, 1, size=(1500, 25)).astype(np.float32)
        X_query = np.random.uniform(-1, 1, size=(5000, 25)).astype(np.float32)
        Y = (add_amount*(np.sin(X[:, 0]) + np.cos(X[:, 1])) + mult_amount*(np.sin(X[:, 2])*np.cos(X[:, 3])))
        Y += np.random.normal(0, 2, size=(1500, ))
        Y = Y.astype(np.float32)
        
        X_train, X_val, Y_train, Y_val = train_test_split(X, Y)
        for dropout_rate in [0.0, 0.125, 0.25, 0.375, 0.5]:
            for fit_iter in range(3):
                model = build_model('relu', dropout_rate, 32)
                epoch = 0
                while epoch < max_epoch:
                    print(dropout_rate, fit_iter, epoch)
                    
                    pred = model(X_query, training=False).numpy()
                    xgb1 = xgb(max_depth=1, n_estimators=1000)
                    xgb2 = xgb(max_depth=2, n_estimators=1000)
                    xgb3 = xgb(max_depth=3, n_estimators=1000)

                    xgb1.fit(X_query, pred)
                    xgb1_preds = xgb1.predict(X_query)
                    residual1 = pred - xgb1_preds
                    
                    xgb2.fit(X_query, pred)
                    xgb2_preds = xgb2.predict(X_query)

                    xgb3.fit(X_query, pred)
                    xgb3_preds = xgb3.predict(X_query)

                    xgb1_var = np.var(xgb1_preds)
                    xgb2_var = np.var(xgb1_preds - xgb2_preds)
                    xgb3_var = np.var(xgb2_preds - xgb3_preds)
                    
                    train_mse = np.mean(np.square(Y_train - np.squeeze(model(X_train, training=False).numpy())))
                    val_mse = np.mean(np.square(Y_val - np.squeeze(model(X_val, training=False).numpy())))
                    
                    print(xgb1_var, xgb2_var, xgb3_var, train_mse, val_mse)
                    print('{:d}\t{:.3f}\t{:.3f}\t{:d}\t{:d}\t{}\t{}\t{}\t{}\t{}'.format(
                                    dataset_iter, add_amount, dropout_rate, fit_iter, epoch,
                        xgb1_var, xgb2_var, xgb3_var, train_mse, val_mse),
                        file=results_file, flush=True)
                    if epoch < 50:
                        model.fit(X_train, Y_train, epochs=5, verbose=1)
                        epoch += 5
                    elif epoch < max_epoch:
                        model.fit(X_train, Y_train, epochs=25, verbose=0)
                        epoch += 25

Building model with relu activation and 0.000 dropout
0.0 0 0
0.03418691 0.0032073674 0.002490637 4.3504786 4.583871
Train on 1125 samples
Epoch 1/5
1125/1125 [==============================] - 0s 421us/sample - loss: 4.1836 - mse: 4.1836
Epoch 2/5
1125/1125 [==============================] - 0s 50us/sample - loss: 3.9783 - mse: 3.9783
Epoch 3/5
1125/1125 [==============================] - 0s 62us/sample - loss: 3.8725 - mse: 3.8725
Epoch 4/5
1125/1125 [==============================] - 0s 101us/sample - loss: 3.7952 - mse: 3.7952
Epoch 5/5
1125/1125 [==============================] - 0s 87us/sample - loss: 3.7183 - mse: 3.7183
0.0 0 5
0.10101339 0.0072883675 0.005400865 3.6596675 4.2597075
Train on 1125 samples
Epoch 1/5
1125/1125 [==============================] - 0s 254us/sample - loss: 3.6532 - mse: 3.6532
Epoch 2/5
1125/1125 [==============================] - 0s 96us/sample - loss: 3.5982 - mse: 3.5982
Epoch 3/5
1125/1125 [==============================] - 0s 81us/sample - loss: 3

0.21843472 0.012178106 0.010635132 3.3715413 4.203711
Train on 1125 samples
Epoch 1/5
1125/1125 [==============================] - 0s 40us/sample - loss: 3.3867 - mse: 3.3867
Epoch 2/5
1125/1125 [==============================] - 0s 38us/sample - loss: 3.3465 - mse: 3.3465
Epoch 3/5
1125/1125 [==============================] - 0s 42us/sample - loss: 3.3247 - mse: 3.3247
Epoch 4/5
1125/1125 [==============================] - ETA: 0s - loss: 3.3047 - mse: 3.304 - 0s 41us/sample - loss: 3.2774 - mse: 3.2774
Epoch 5/5
1125/1125 [==============================] - 0s 46us/sample - loss: 3.2391 - mse: 3.2391
0.0 1 15
0.2920759 0.01828909 0.015767355 3.1751611 4.2471457
Train on 1125 samples
Epoch 1/5
1125/1125 [==============================] - 0s 61us/sample - loss: 3.1982 - mse: 3.1982
Epoch 2/5
1125/1125 [==============================] - 0s 59us/sample - loss: 3.1661 - mse: 3.1661
Epoch 3/5
1125/1125 [==============================] - 0s 58us/sample - loss: 3.1258 - mse: 3.1258
Epoch 4/5


1125/1125 [==============================] - 0s 67us/sample - loss: 2.9327 - mse: 2.9327
Epoch 2/5
1125/1125 [==============================] - 0s 50us/sample - loss: 2.8963 - mse: 2.8963
Epoch 3/5
1125/1125 [==============================] - 0s 48us/sample - loss: 2.8536 - mse: 2.8536
Epoch 4/5
1125/1125 [==============================] - 0s 52us/sample - loss: 2.8107 - mse: 2.8107
Epoch 5/5
1125/1125 [==============================] - 0s 66us/sample - loss: 2.7707 - mse: 2.7707
0.0 2 25
0.32555112 0.047699414 0.042585287 2.6945086 4.2887645
Train on 1125 samples
Epoch 1/5
1125/1125 [==============================] - 0s 75us/sample - loss: 2.7240 - mse: 2.7240
Epoch 2/5
1125/1125 [==============================] - 0s 57us/sample - loss: 2.6840 - mse: 2.6840
Epoch 3/5
1125/1125 [==============================] - 0s 57us/sample - loss: 2.6588 - mse: 2.6588
Epoch 4/5
1125/1125 [==============================] - 0s 62us/sample - loss: 2.6100 - mse: 2.6100
Epoch 5/5
1125/1125 [============

1125/1125 [==============================] - 0s 57us/sample - loss: 3.3683 - mse: 3.3683
Epoch 2/5
1125/1125 [==============================] - 0s 54us/sample - loss: 3.3123 - mse: 3.3123
Epoch 3/5
1125/1125 [==============================] - 0s 52us/sample - loss: 3.2718 - mse: 3.2718
Epoch 4/5
1125/1125 [==============================] - 0s 54us/sample - loss: 3.3740 - mse: 3.3740
Epoch 5/5
1125/1125 [==============================] - 0s 53us/sample - loss: 3.3176 - mse: 3.3176
0.125 1 45
0.20687456 0.020325487 0.018560547 2.9661188 4.136082
Train on 1125 samples
Epoch 1/5
1125/1125 [==============================] - 0s 52us/sample - loss: 3.2840 - mse: 3.2840
Epoch 2/5
1125/1125 [==============================] - 0s 53us/sample - loss: 3.2396 - mse: 3.2396
Epoch 3/5
1125/1125 [==============================] - 0s 53us/sample - loss: 3.2234 - mse: 3.2234
Epoch 4/5
1125/1125 [==============================] - 0s 56us/sample - loss: 3.2474 - mse: 3.2474
Epoch 5/5
1125/1125 [===========

0.23009007 0.033781156 0.0314988 2.6502213 4.158754
0.125 2 100
0.20635512 0.041453883 0.035569485 2.427912 4.184235
0.125 2 125
0.20002113 0.043845158 0.04191437 2.229736 4.237162
0.125 2 150
0.20712371 0.051955774 0.05073486 2.0475526 4.32006
0.125 2 175
0.21127354 0.055083532 0.051137976 1.9026302 4.37145
0.125 2 200
0.19654243 0.05782954 0.05168313 1.80881 4.3860145
0.125 2 225
0.17237878 0.05652826 0.055682957 1.7445693 4.407841
0.125 2 250
0.19351985 0.056568958 0.05884784 1.6731434 4.365589
0.125 2 275
0.18472476 0.05875873 0.066162705 1.5789185 4.4491167
0.125 2 300
0.16066693 0.059081018 0.056907643 1.5500829 4.473304
0.125 2 325
0.19060063 0.060224272 0.059816796 1.4775199 4.473453
0.125 2 350
0.19657136 0.06594061 0.060679786 1.4146485 4.470878
0.125 2 375
0.17213257 0.06287635 0.065763734 1.3772448 4.471309
0.125 2 400
0.18017 0.063772 0.06824822 1.3588325 4.4476023
0.125 2 425
0.17814453 0.06159881 0.0651617 1.3112602 4.446019
0.125 2 450
0.19105703 0.06382444 0.06430163 1

...

0.1535888 0.05156567 0.049149428 1.5879792 4.4521236
0.125 2 325
0.19851944 0.056228075 0.053256933 1.5027804 4.620239
0.125 2 350
0.18098924 0.052744567 0.05337289 1.4719192 4.672629
0.125 2 375
0.16254714 0.05916286 0.055218987 1.419339 4.6361384
0.125 2 400
0.18065476 0.061413184 0.057295788 1.3624042 4.647802
0.125 2 425
0.17414512 0.05855594 0.059480134 1.3182044 4.58669
0.125 2 450
0.17372045 0.057062104 0.056554224 1.3277705 4.4993987
0.125 2 475
0.17890306 0.059817236 0.06062987 1.2727498 4.5622945
Building model with relu activation and 0.250 dropout
0.25 0 0
0.048418444 0.004368585 0.0029347718 4.0900702 4.2575483
Train on 1125 samples
Epoch 1/5
1125/1125 [==============================] - 0s 274us/sample - loss: 4.1705 - mse: 4.1705
Epoch 2/5
1125/1125 [==============================] - 0s 34us/sample - loss: 4.1331 - mse: 4.1331
Epoch 3/5
1125/1125 [==============================] - 0s 34us/sample - loss: 4.1449 - mse: 4.1449
Epoch 4/5
1125/1125 [===========================

1125/1125 [==============================] - 0s 41us/sample - loss: 3.9387 - mse: 3.9387
Epoch 5/5
1125/1125 [==============================] - 0s 43us/sample - loss: 3.9281 - mse: 3.9281
0.25 2 15
0.061551604 0.0032744515 0.0020193467 3.820724 4.1916595
Train on 1125 samples
Epoch 1/5
1125/1125 [==============================] - 0s 42us/sample - loss: 4.0166 - mse: 4.0166
Epoch 2/5
1125/1125 [==============================] - 0s 41us/sample - loss: 3.9463 - mse: 3.9463
Epoch 3/5
1125/1125 [==============================] - 0s 41us/sample - loss: 3.9178 - mse: 3.9178
Epoch 4/5
1125/1125 [==============================] - 0s 47us/sample - loss: 3.9189 - mse: 3.9189
Epoch 5/5
1125/1125 [==============================] - 0s 48us/sample - loss: 3.9756 - mse: 3.9756
0.25 2 20
0.07317811 0.0035178005 0.0023217998 3.7761784 4.1664343
Train on 1125 samples
Epoch 1/5
1125/1125 [==============================] - 0s 42us/sample - loss: 3.8714 - mse: 3.8714
Epoch 2/5
1125/1125 [===================

1125/1125 [==============================] - 0s 51us/sample - loss: 4.0100 - mse: 4.0100
Epoch 3/5
1125/1125 [==============================] - 0s 57us/sample - loss: 4.0155 - mse: 4.0155
Epoch 4/5
1125/1125 [==============================] - 0s 50us/sample - loss: 3.9787 - mse: 3.9787
Epoch 5/5
1125/1125 [==============================] - 0s 47us/sample - loss: 4.0321 - mse: 4.0321
0.375 1 35
0.0228867 0.0010638464 0.0007392555 3.9176502 4.159488
Train on 1125 samples
Epoch 1/5
1125/1125 [==============================] - 0s 57us/sample - loss: 3.9629 - mse: 3.9629
Epoch 2/5
1125/1125 [==============================] - 0s 45us/sample - loss: 3.9481 - mse: 3.9481
Epoch 3/5
1125/1125 [==============================] - 0s 47us/sample - loss: 3.9525 - mse: 3.9525
Epoch 4/5
1125/1125 [==============================] - 0s 43us/sample - loss: 4.0359 - mse: 4.0359
Epoch 5/5
1125/1125 [==============================] - 0s 45us/sample - loss: 4.0320 - mse: 4.0320
0.375 1 40
0.023105742 0.001166

0.0040383786 0.00020649999 0.00016470575 4.033863 4.2124434
0.5 0 75
0.005662379 0.0001995628 0.00013146427 4.011575 4.2033668
0.5 0 100
0.010513874 0.00038699064 0.00024520792 3.9670699 4.2128706
0.5 0 125
0.010872328 0.00038350408 0.00024329525 3.9527621 4.1985283
0.5 0 150
0.015516379 0.0006806056 0.00037801222 3.9197552 4.1914253
0.5 0 175
0.01990517 0.00082228414 0.00048645 3.890981 4.1849117
0.5 0 200
0.014659462 0.0006280044 0.00042339164 3.9008324 4.199204
0.5 0 225
0.0146785155 0.0007295618 0.00054300996 3.8857353 4.1902137
0.5 0 250
0.013723794 0.0008050523 0.00062646496 3.8762486 4.193377
0.5 0 275
0.015697293 0.0010752241 0.00078800536 3.848453 4.1781673
0.5 0 300
0.016618488 0.001412542 0.0009772162 3.8258185 4.1787496
0.5 0 325
0.019886354 0.0013402277 0.0010632008 3.8079736 4.1795306
0.5 0 350
0.013469403 0.0010937102 0.00078732846 3.8435516 4.189919
0.5 0 375
0.015848469 0.0013823843 0.0009819161 3.810707 4.1765847
0.5 0 400
0.015268314 0.0013156765 0.0010420299 3.80912

0.5 1 450
0.012964378 0.0013170547 0.0010217763 3.80381 4.181562
0.5 1 475
0.013058115 0.0014635447 0.0011601541 3.7895443 4.1758075
Building model with relu activation and 0.500 dropout
0.5 2 0
0.038341973 0.0035022406 0.002469839 4.1124372 4.279432
Train on 1125 samples
Epoch 1/5
1125/1125 [==============================] - 0s 270us/sample - loss: 4.6678 - mse: 4.6678
Epoch 2/5
1125/1125 [==============================] - 0s 42us/sample - loss: 4.2821 - mse: 4.2821
Epoch 3/5
1125/1125 [==============================] - 0s 40us/sample - loss: 4.3575 - mse: 4.3575
Epoch 4/5
1125/1125 [==============================] - 0s 39us/sample - loss: 4.2202 - mse: 4.2202
Epoch 5/5
1125/1125 [==============================] - 0s 42us/sample - loss: 4.1914 - mse: 4.1914
0.5 2 5
0.007258709 0.0007243836 0.00064325234 4.054098 4.2512193
Train on 1125 samples
Epoch 1/5
1125/1125 [==============================] - 0s 45us/sample - loss: 4.2180 - mse: 4.2180
Epoch 2/5
1125/1125 [=======================